##### QnA chatcot using RAG. context is standford lectures. this is a take home assignment for an application(Ema)

In [7]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


# Data Ingestion

In [27]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader1=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/introduction/",),
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap")))) 
loader2=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/legality/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
loader3=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/modeling/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main_content-wrap"))))
loader4=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/training/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
### we are using 4 lectures: intro, legality, modeling and training. 
### since all the content was inside the main class "main-content-wrap" we used that


In [36]:
loaders=[loader1,loader2,loader3,loader4] #array of loaders to work upon

In [37]:
docs=[]
for loader in loaders:
    docs.extend(loader.load()) #loading documents together

# Transform

In [71]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=10000)

In [72]:
split=splitter.split_documents(docs) # splitting documents into chunks 

In [73]:
len(split)

8

In [74]:
split[0].page_content

'LecturesIntroduction \\[\\newcommand{\\sV}{\\mathcal{V}} \\newcommand{\\nl}[1]{\\textsf{#1}} \\newcommand{\\generate}[1]{\\stackrel{#1}{\\rightsquigarrow}}\\]Welcome to CS324! This is a new course on understanding and developing large language models.What is a language model?A brief historyWhy does this course exist?Structure of this course  What is a language model?The classic definition of a language model (LM) is a probability distribution over sequences of tokens. Suppose we have a vocabulary \\(\\sV\\) of a set of tokens. A language model \\(p\\) assigns each sequence of tokens \\(x_1, \\dots, x_L \\in \\sV\\) a probability (a number between 0 and 1):\\[p(x_1, \\dots, x_L).\\]The probability intuitively tells us how “good” a sequence of tokens is. For example, if the vocabulary is \\(\\sV = \\{ \\nl{ate}, \\nl{ball}, \\nl{cheese}, \\nl{mouse}, \\nl{the} \\}\\), the language model might assign (demo):\\[p(\\nl{the}, \\nl{mouse}, \\nl{ate}, \\nl{the}, \\nl{cheese}) = 0.02,\\] \\[p(

# Retriever

In [66]:
from langchain_core.prompts import ChatPromptTemplate

system_message=(""" You are a question-answer assistant.
                    Answer the questions based on the context provided.
                    Keep the answers as accurate and concise as possible.
                    If the answer is not present in the context then say its not available in the context.
                    \n\n
                    {context}.""")

prompt=ChatPromptTemplate.from_messages([("system",system_message),
                                        ("human","{input}")])


In [67]:
from langchain_community.llms import Ollama

model=Ollama(model="llama3",num_gpu=5)

In [68]:
from langchain.chains.combine_documents import create_stuff_documents_chain

stuff_doc_chain=create_stuff_documents_chain(model,prompt) #creating stuff document chain or basically chaining llm and prompt

In [89]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

vector_storage=Chroma(embedding_function=OllamaEmbeddings(),persist_directory="/vector_store") #initializing our vector storage

In [76]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore

In [90]:
retriever=MultiVectorRetriever(vectorstore=vector_storage, byte_store=InMemoryByteStore())

In [91]:
child_splitter=RecursiveCharacterTextSplitter(chunk_size=200)

In [92]:
sub_docs=[]

for i, doc in enumerate(docs):
    sub_doc=child_splitter.split_documents([doc])
    sub_docs.extend(sub_doc)

In [93]:
len(sub_docs[1].page_content)

195

In [ ]:
retriever.vectorstore.add_documents(sub_docs)

In [ ]:
vector_storage.persist()